In [7]:
# Importing necessary libraries to scrape
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import PIL              # for images
import wordcloud        # ploting word cloud
import requests         # grab web-page
import pickle           # save file
from bs4 import BeautifulSoup as bsopa  # parse web-page
import datetime         # format date/time
from collections import defaultdict
import re               # regular expressions

driver = webdriver.Chrome()  

In [9]:
skill = input('Enter your Skill: ').strip()
place = input('Enter the location: ').strip()
no_of_pages = int(input('Enter the # of pages to scrape: '))
indeed_posts=[]

jobs=[]
company=[]
links=[]
salary=[]
post_date=[]

for page in range(no_of_pages):
    
    # Connecting to Indeed
        target = 'https://www.indeed.com/jobs?q=' + skill + '&l=' + place + '&sort=date' +'&start='+ str(page * 10)
        driver.get(target)
    
        # Scrapping the Web (you can use 'html' or 'lxml')
        soup = bsopa(driver.page_source, "html")

        # Outer Most Entry Point of HTML:
        outer_most_point=soup.find('div',attrs={'id': 'mosaic-provider-jobcards'})
        
        # "UL" lists where the data are stored:

        for i in outer_most_point.find('ul'):
            
        # Job Title:
        
            job_title=i.find('h2',{'class':'jobTitle css-1h4a4n5 eu4oa1w0'})
            if job_title != None:
                jobs=job_title.find('a').text

        # Company Name:
    
            if i.find('span',{'class':'companyName'}) != None:
                company=i.find('span',{'class':'companyName'}).text   
                
        # Links: these Href links will take us to full job description
        
            if i.find('a') != None:
                links=i.find('a',{'class':'jcs-JobTitle'})['href']
                
        # Salary if available:
        
            if i.find('div',{'class':'attribute_snippet'}) != None:
                salary=i.find('div',{'class':'attribute_snippet'}).text

            else:
                salary='No Salary'

        # Job Post Date:

            if i.find('span', attrs={'class': 'date'}) != None:
                post_date = i.find('span', attrs={'class': 'date'}).text

        # Put everything together in a list of lists for the default dictionary
                        
            indeed_posts.append([company,jobs,links,salary, post_date])
            
            
# put together in list

# (create a dictionary with keys and a list of values from above "indeed_posts")

indeed_dict_list=defaultdict(list)

# Fields for our DF 

indeed_spec=['Company','job','link','Salary','Job_Posted_Date']


Enter your Skill: data analyst
Enter the location: California
Enter the # of pages to scrape: 1


In [10]:
indeed_posts

[['PG&E Corporation',
  [],
  '/pagead/clk?mo=r&ad=-6NYlbfkN0Dl5O3UwlcwwCSNUOo_pIXFXhqhPgZDNLRFp2hAbMlfu8fEhSbyhvBx-IxrRTMlcWyzu8rgZvsHFsOLg5X-YTnfCYMBTwjv4uTmFPRxdg5vgldB5QbR7Nfi5iNenOv7bYE87fDeO4a5CBhEh3KhVU9-Mh3UnKM6vUlZ7MXhRALUaWtD19Z-QZ0O7-Bsk08USAEifJxWz2sNX2z86DTFLR7XudrKJw89VdmNApqQixnyRSFwlFrIqd72c6Ky0CcaSuNAp7sEQhL5oCfN6EAyPVY5FJ9j-QeBsTPg1WpbY655BqjdHB7JOI8xVv0_22wCvaIKy1Ts4PJips9wjP_9pdm9uhOIFrtnoQDB-Eqehc5dUOv0S_UxW5FwR5vtpe5g2XjNuUoa0qzC10BziIa4-9nJZpdjwPaae4nxYDrNpHpg9Jf9jG6KXY2MlPBdoB-n8BNNxoGRtuC8MA==&xkcb=SoD9-_M3TDFOcnX-3Z0LbzkdCdPP&p=0&fvj=0&vjs=3',
  'No Salary',
  'PostedJust posted'],
 ['KARL STORZ',
  'Data Visualization Analyst - Contract',
  '/rc/clk?jk=e146f260381ad30b&fccid=7583a919bc1f961c&vjs=3',
  '$50 - $60 an hour',
  'PostedJust posted'],
 ['Fox Corporation',
  'Senior Data Analyst',
  '/rc/clk?jk=4d035532f77ae830&fccid=61c19e057c54ec35&vjs=3',
  'Full-time',
  'PostedJust posted'],
 ['University of California - Irvine',
  'Academic Personnel Business 

In [11]:
print('These Href links will go to a new page containing full job description')
print('\n')
print(pd.DataFrame(indeed_posts,columns=indeed_spec)['link'][0]) 
#these are not the same, probably from recruiter(s)
print(pd.DataFrame(indeed_posts,columns=indeed_spec)['link'][1])

pd.DataFrame(indeed_posts,columns=indeed_spec)

These Href links will go to a new page containing full job description


/pagead/clk?mo=r&ad=-6NYlbfkN0Dl5O3UwlcwwCSNUOo_pIXFXhqhPgZDNLRFp2hAbMlfu8fEhSbyhvBx-IxrRTMlcWyzu8rgZvsHFsOLg5X-YTnfCYMBTwjv4uTmFPRxdg5vgldB5QbR7Nfi5iNenOv7bYE87fDeO4a5CBhEh3KhVU9-Mh3UnKM6vUlZ7MXhRALUaWtD19Z-QZ0O7-Bsk08USAEifJxWz2sNX2z86DTFLR7XudrKJw89VdmNApqQixnyRSFwlFrIqd72c6Ky0CcaSuNAp7sEQhL5oCfN6EAyPVY5FJ9j-QeBsTPg1WpbY655BqjdHB7JOI8xVv0_22wCvaIKy1Ts4PJips9wjP_9pdm9uhOIFrtnoQDB-Eqehc5dUOv0S_UxW5FwR5vtpe5g2XjNuUoa0qzC10BziIa4-9nJZpdjwPaae4nxYDrNpHpg9Jf9jG6KXY2MlPBdoB-n8BNNxoGRtuC8MA==&xkcb=SoD9-_M3TDFOcnX-3Z0LbzkdCdPP&p=0&fvj=0&vjs=3
/rc/clk?jk=e146f260381ad30b&fccid=7583a919bc1f961c&vjs=3


,Company,job,link,Salary,Job_Posted_Date
0,PG&E Corporation,[],/pagead/clk?mo=r&ad=-6NYlbfkN0Dl5O3UwlcwwCSNUO...,No Salary,PostedJust posted
1,KARL STORZ,Data Visualization Analyst - Contract,/rc/clk?jk=e146f260381ad30b&fccid=7583a919bc1f...,$50 - $60 an hour,PostedJust posted
2,Fox Corporation,Senior Data Analyst,/rc/clk?jk=4d035532f77ae830&fccid=61c19e057c54...,Full-time,PostedJust posted
3,University of California - Irvine,Academic Personnel Business Analyst - Hybrid/R...,/rc/clk?jk=9a3cdebf3fdd75f4&fccid=eacce60f6c6b...,Full-time,PostedJust posted
4,"Vanir Construction Management, Inc.",Business Analyst I,/rc/clk?jk=7f43bcfe5bd443ed&fccid=0cf385a4d66a...,"$65,000 - $85,000 a year",PostedJust posted
5,"Vanir Construction Management, Inc.",Business Analyst I,/rc/clk?jk=7f43bcfe5bd443ed&fccid=0cf385a4d66a...,No Salary,PostedJust posted
6,Samsara,Senior Reliability Data Analyst,/rc/clk?jk=46eb9cfca85ce9d7&fccid=4a9c3fc37e5f...,No Salary,PostedJust posted
7,TRUCONNECT COMMUNICATIONS INC,Associate Business Analyst,/rc/clk?jk=6be0fdd1a9ba6dc9&fccid=daab0be90981...,No Salary,PostedJust posted
8,SBCS Corporation,Contract and Data Analyst (PT),/rc/clk?jk=bddfd7fa6f5c3afb&fccid=730e4ec11724...,$31.25 an hour,PostedJust posted
9,Wealthfront,FP&A Analyst,/rc/clk?jk=62f4d9871f2fb5a1&fccid=90344e51e9e0...,"$80,000 - $120,000 a year",PostedJust posted


In [13]:
job_descr_txt=[]

# Indeed DF with columns we made above and the stored data from scraping
Indeed_DF = pd.DataFrame(indeed_posts,columns=indeed_spec)

# convert Series to lists of strings
my_super_fun_Indeed_links = list(Indeed_DF['link'])

# iterator will be our index value for default_dict_list
for i in range(len(my_super_fun_Indeed_links)):
        
    url_href='https://indeed.com' + my_super_fun_Indeed_links[i]
#     print(url_href)

    target2 = url_href
    driver.get(target2)
    soup_ = bsopa(driver.page_source, 'html')
    
    for ii in soup_.find('div',{'class':'jobsearch-jobDescriptionText'}):
        try:
            job_descr_txt.append([i,''.join(ii.text.strip())])
        except AttributeError:
            job_descr_txt.append([i,''])
job_descr_txt

[[0, 'Job Location:'],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, 'Responsibilities typically include many or all the following:'],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, 'Data Analysis:'],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, 'Metrics and Reporting:'],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, 'Budgeting & Forecasting:'],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, 'Research:'],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, 'Quality Assurance & Business Process Improvement:'],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, 'Project Management:'],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, 'A reasonable salary range is:'],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, 'Responsibilities:'],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0, ''],
 [0

In [15]:
# create dictionary with values as lists
dct_lst= defaultdict(list)

for i in job_descr_txt:
    #key value pairs for default_dict_list
    dct_lst[i[0]].append(i[1])
    
dict_lst_jobsDescr=[]

for i in dct_lst.values(): # string join: lists of lists of strings
    dict_lst_jobsDescr.append(''.join(i))
    
dict_lst_jobsDescr[1]

'Company: KARL STORZ Imaging, Inc. (KSI)\n Job Code: 13216\n Pay Grade: US-C16\n Salary Range: $50hr - $60/hr\n\n\n\n\n\n\n\n\nDescription\n KARL STORZ SE & Co. KG based in Tuttlingen, Germany, is a family-owned, global company committed to benefiting humanity by advancing medical technology through innovation and education.\n For more than 75 years, KARL STORZ has been dedicated to earning its international reputation as a leader that designs, engineers, manufactures, and markets all its products with an emphasis on visionary design, precision craftsmanship, and clinical effectiveness.\n \n\nJOB SUMMARY: Brief description of what this person does, in simple direct terms\n\n Works with product development management to efficiently translate statistical data in ways that are useful for both subject matter experts as well as business users. Perform as a statistical translator to take often complex combinations of data that may be very difficult to conceptualize and present it in ways tha

In [16]:
Indeed_DF['text_descrption'] = dict_lst_jobsDescr
Indeed_DF.head()

,Company,job,link,Salary,Job_Posted_Date,text_descrption
0,PG&E Corporation,[],/pagead/clk?mo=r&ad=-6NYlbfkN0Dl5O3UwlcwwCSNUO...,No Salary,PostedJust posted,Job Location:Responsibilities typically includ...
1,KARL STORZ,Data Visualization Analyst - Contract,/rc/clk?jk=e146f260381ad30b&fccid=7583a919bc1f...,$50 - $60 an hour,PostedJust posted,"Company: KARL STORZ Imaging, Inc. (KSI)\n Job ..."
2,Fox Corporation,Senior Data Analyst,/rc/clk?jk=4d035532f77ae830&fccid=61c19e057c54...,Full-time,PostedJust posted,OVERVIEW OF THE COMPANY\n Fox Corporation\n ...
3,University of California - Irvine,Academic Personnel Business Analyst - Hybrid/R...,/rc/clk?jk=9a3cdebf3fdd75f4&fccid=eacce60f6c6b...,Full-time,PostedJust posted,Come join our energized and effective team com...
4,"Vanir Construction Management, Inc.",Business Analyst I,/rc/clk?jk=7f43bcfe5bd443ed&fccid=0cf385a4d66a...,"$65,000 - $85,000 a year",PostedJust posted,About Vanir: \n \n Vanir has been exceeding ...
